First, import all the libraries (from other folders)

In [32]:
#algorithm imports
from Algorithms import *

#arm imports
from Arms import *

#plot imports
from Plots import *

import numpy as np
import random as rand
import matplotlib.pyplot as plt

Next, we want to define the function to run the algorithm. This function takes in a specified algorithm as well as a time horizon.

In [34]:
# horizon is total # of steps
def run_algorithm(algorithm, horizon, arms, epoch):
    total_reward = 0
    oracle_total_reward = 0
    oracle = Oracle(arms)
    # create a matrix of nxm
    # n - # of arms
    # m - # of steps (measures each mean at each step, used for plotting later)
    mean_history = [[] for i in range(len(arms))]
    regret_history = []

    for t in range (horizon):
        # ONLY FOR BROWNIAN ARMS
        if t != 0 and t % epoch == 0:
            print(f"Algorithm: {algorithm.select_arm()}")
            print(f"Oracle: {oracle.select_arm()}")
            algorithm.reinitialize(n_arms=len(arms))
            print("Reinitialized!")

        for arm in arms:
            arm.simulate_brownian()
        #chosen arm is index
        chosen_arm = algorithm.select_arm()
        reward = arms[chosen_arm].draw()
        total_reward += reward

        #oracle action
        oracle_chosen_arm = oracle.select_arm()
        oracle_reward = arms[oracle_chosen_arm].draw()
        oracle_total_reward += oracle_reward
        algorithm.update(chosen_arm, reward)

        #used for plotting regret
        if t == 0:
            regret_history.append(oracle_reward - reward)
        else:
            regret_history.append(oracle_reward - reward + regret_history[t - 1])

        #used for plotting arm means
        for i in range(len(arms)):
            mean_history[i].append(arms[i].mean)
    #plot regret vs log rounds
    plot_regret_log_t(regret_history)
    plot_arm_means(mean_history)


Now, we define a list of arms. These are the arms are algorithm will pick from. We also will generate the arms depending on the type (can be changed so that they are not all the same type).

In [35]:
#horizon is # of steps or rounds that alg will run
arms = []
#n - num of arms
n = 5
#generate each arm based on Arm Type and n # of arms

expected_vals = [rand.random() for i in range(n)]
std_dev = 0.1
drift = 0
volatility = 0.01
for val in expected_vals:
    arms.append(BrownianArm(val, std_dev, drift, volatility))


Something to note is that for certain algorithms, (for example, epsilon greedy), you need to define different paramters. For all algorithms, you need to define the number of arms (n)

In [38]:
eps = 0.01
horizon = 5000
#Algorithms
eps_algo = EpsilonGreedy(eps, n_arms = len(arms))
#UCB_algo = UCB1(n_arms = len(arms))
run_algorithm(eps_algo, horizon, arms, epoch=50)

TypeError: 'module' object is not callable